<a href="https://colab.research.google.com/github/sanjayparajuli/colab/blob/main/sqlite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pysqlite3
from pysqlite3 import dbapi2 as sqlite3
print(sqlite3.sqlite_version)
import pandas as pd
from IPython.display import display, HTML

3.37.2


In [3]:
dbname = 'music_streaming_1.db'
def printSqlResults(cursor, tblName):
  try:
    df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
    display(HTML("<b><font color=Green> " + tblName + "</font></b>" + df.to_html(index=False)))
  except:
    pass

def runSql(caption, query):
  conn = sqlite3.connect(dbname)
  cursor = conn.cursor()
  cursor.execute(query)
  printSqlResults(cursor, caption)
  conn.close()

def runStepByStepSql(query, fromline):
  lines = query.strip().split('\n')
  for lineindx in range(fromline, len(lines)):
    partial_query = '\n'.join(lines[:lineindx])
    caption = 'Query till line:' + partial_query
    runSql(caption, partial_query + ':')
